# Imports

In [ ]:
from autoboltagent import verbose_prompts, grammars
from autoboltagent  import prompts
from autoboltagent.verbose_agents import VerboseLowFidelityAgent
from autoboltagent.agents import LowFidelityAgent
from autoboltagent.tools.logger import AgentLogger
from autoboltagent.VLLMModelCustom import VLLMModelCustom

In [ ]:
from vllm import SamplingParams
from vllm.sampling_params import StructuredOutputsParams
import smolagents

# Setup

### set up logger and db

In [ ]:
db_url = "sqlite:///../src/agent_logs_grammar_prod.db"
logger = AgentLogger(db_url)

### params

In [ ]:
joint_configuration = {
    "load": 60000,
    "desired_safety_factor": 3.0,
    "bolt_yield_strength": 940,
    "plate_yield_strength": 250,
    "preload": 150000,
    "pitch": 1.5,
    "plate_thickness": 10,
    "bolt_elastic_modulus": 210,
    "plate_elastic_modulus": 210
    }

input = """{
    "load": 60000,
    "desired_safety_factor": 3.0,
    "bolt_yield_strength": 940,
    "plate_yield_strength": 250,
    "preload": 150000,
    "pitch": 1.5,
    "plate_thickness": 10,
    "bolt_elastic_modulus": 210,
    "plate_elastic_modulus": 210
    }"""

grammar_sop = StructuredOutputsParams(
    grammar=grammars.low_fidelity_agent_grammar_debug
)

sampling_params = SamplingParams(
    max_tokens=200,
    temperature=0.0,
    structured_outputs=grammar_sop
)

# Models

### Local

In [ ]:
model = smolagents.VLLMModel(
    model_id="RedHatAI/Qwen2.5-3B-Instruct-quantized.w8a8",
    model_kwargs={
        "gpu_memory_utilization": 0.85,
    },
)

In [ ]:
# Custom local VLLM model with grammar

model = VLLMModelCustom(
        model_id="RedHatAI/Qwen2.5-3B-Instruct-quantized.w8a8",
        apply_chat_template_kwargs=None,
        model_kwargs={
            "gpu_memory_utilization": 0.85,
        },
        sampling_params=sampling_params,
    )

### Cloud models

In [ ]:
FIREWORKS_API_KEY = ""

In [ ]:
# smolagents InferenceClientModel

model = smolagents.InferenceClientModel( # type: ignore
    provider="fireworks-ai",
    model_id="openai/gpt-oss-20b",
    token=FIREWORKS_API_KEY,
)

In [ ]:
model = smolagents.OpenAIServerModel(
    model_id="accounts/fireworks/models/gpt-oss-20b",
    api_base="https://api.fireworks.ai/inference/v1", 
    api_key=FIREWORKS_API_KEY,
    # response_format={
    #     "type": "grammar",
    #     "grammar": grammars.low_fidelity_agent_grammar_debug
    # }
)

# Run agent

### Single agent run

In [ ]:
agent = VerboseLowFidelityAgent(model, joint_configuration, "verbose low fidelity agent", f"verbose prompts + minimized + reason 512 + gpt 11", 3.0, logger, max_steps=100)
instruction = verbose_prompts.EXAMPLE_TASK_INSTRUCTIONS.format(input)
agent.run(instruction)

### Loop agent run

In [ ]:
for i in range(0,25):
        agent = VerboseLowFidelityAgent(model, joint_configuration, "verbose low fidelity agent", f"verbose prompts + minimized + reason 512 + gpt {i}", 3.0, logger, max_steps=100)
        instruction = verbose_prompts.EXAMPLE_TASK_INSTRUCTIONS.format(input)
        agent.run(instruction)
